# Practica Final

Aldao Amoedo, Héctor

Cabaleiro Pintos, Laura

Cotardo Valcárcel, Donato José

Romero Conde, José

---

## Preparación de los datos

El punto 1, está completado, aunque creo que el 1.3, donde preparamos los datos transformandolos y rellanando los nulos está mal.

### 1. Cargar los datos y descripción

In [ ]:
using CSV
using DataFrames

df = CSV.read("Datos_Practica_Evaluacion_1.csv", DataFrame)

num_instancias, num_variables = size(df)
num_individuos = length(unique(df[:, 1]))
num_clases_salida = length(unique(df[:, end]))

println("Número de variables: $num_variables")
println("Número de instancias: $num_instancias")
println("Número de individuos: $num_individuos")
println("Número de clases de salida: $num_clases_salida")

Los datos dados ya están cargados, y como podemos observar tienen:
* 563 Variables
* 10299 Instancias
* 30 Individuos
* 6 Clases de salida

### 2. Calcular porcentaje de nulos

In [ ]:
println("Porcentaje de nulos por variable:")
num_nulos_totales = 0
for col ∈ names(df)
    num_nulos = count(ismissing, df[:, col])
    num_nulos_totales += num_nulos
    porcentaje_nulos = (num_nulos / num_instancias) * 100
    println("$col: $porcentaje_nulos% nulos")
end

porcentaje_nulos_totales = (num_nulos_totales / (num_instancias * num_variables)) * 100
println("Porcentaje total de nulos en el conjunto: $porcentaje_nulos_totales%")

### 3. Preparar los datos para las técnicas de clasificación

Para rellenar valores faltantes tenemos que hacernos una idea de que tipo de datos encontraremos.

In [ ]:
Set([eltype(df[!,col]) for col in names(df)])

In [ ]:
sum([eltype(df[!,col])==String31 for col in names(df)]) # la ultima

In [ ]:
sum([eltype(df[!,col])==Int64 for col in names(df)]) # la primera

Es decir, todas las columnas que tienen valores faltantes son de tipo númerico

In [ ]:
using Statistics
using StatsBase

for col ∈ names(df)
    if eltype(df[!, col]) == Union{Missing, Float64}
        media_columna = mean(skipmissing(df[!, col]))
        cuantos_nulos = sum(ismissing.(df[!, col]))
        println("Columna $col reemplaza a sus $cuantos_nulos nulos por $media_columna")
        df[ismissing.(df[!, col]), col] .= media_columna
        df[!, col] = Float64.(df[!, col]) 
    end
end

println("Valores nulos rellenados.")

Convertimos variables categóricas a numéricas usando One-Hot Encoding

In [ ]:
clases = String[]#se usará más tarde
for col in names(df)
    if eltype(df[!, col]) == String31
        categorias = unique(df[!, col])
        for categoria in categorias
            nombre = "$(col)_$(categoria)"
            push!(clases, nombre)
            nueva_columna = Symbol(nombre)
            df[!, nueva_columna] = df[!, col] .== categoria
        end
        select!(df, Not(col)) # eliminamos la columna original
    end
end

println("One-Hot Encoding aplicado exitosamente.")


Validamos los datos

In [ ]:
println("Validación de datos:")

# Calcular la cantidad total de valores nulos
total_nulos = sum(col -> count(ismissing, col), eachcol(df))
println("Valores nulos restantes: $total_nulos")

# Verificar tipo de datos de cada columna
println("Tipos de datos por columna:")
for col in names(df)
    println("$col: $(eltype(df[!, col]))")
end

In [ ]:
CSV.write("DatosPreprocesados.csv",df) # por si mas tarde se quieren importar directamente

In [ ]:
# df = CSV.read("DatosPreprocesados.csv", DataFrame)

### 4. Segmentar el 10% de los datos usando HoldOut

In [ ]:
using Random
Random.seed!(172)

individuos = unique(df[:, :subject])  # Extraer individuos

num_holdout = Int(round(0.1 * length(individuos)))  # 10% de los individuos
holdout_individuos = shuffle(individuos)[1:num_holdout]  # Seleccionar individuos aleatorios

holdout_df = filter(fila -> fila.subject in holdout_individuos, df)
train_df = filter(fila -> !(fila.subject in holdout_individuos), df)

println("Individuos en el holdout: ", holdout_individuos)
println("Tamaño del conjunto de entrenamiento: $(size(train_df)[1])")
println("Tamaño del conjunto de holdout: $(size(holdout_df)[1])")


In [ ]:
holdout_subjects = unique(holdout_df[:, :subject])
train_subjects = unique(train_df[:, :subject])

intersection = intersect(holdout_subjects, train_subjects)

if isempty(intersection)
    println("La división es correcta: no hay individuos compartidos entre los conjuntos.")
else
    println("Error: Hay individuos compartidos entre los conjuntos.")
    println("Individuos compartidos: $intersection")
end

### 5. Hacer 5 fold cross-validation

In [ ]:
n_folds = 5

fold_indices = randperm(length(individuos))  # Permutar aleatoriamente los índices
fold_tamano = Int(round(length(individuos) / n_folds))
folds = [individuos[fold_indices[(i - 1) * fold_tamano + 1:min(i * fold_tamano, end)]] for i in 1:n_folds]

# Crear folds a nivel de instancias
fold_data = []
for fold in folds
    push!(fold_data, filter(fila -> fila.subject in fold, df)[!,2:end]) # quitamos la columna subject que ya no nos importa
    #print(filter(fila -> fila.subject in fold, df)[1,1])
end

# Imprimir resumen de los folds
for (i, fold) in enumerate(fold_data)
    println("Fold $i:")
#    println("Número de participantes: $(length(unique(fold[:, :subject])))")
    println("Número de instancias: $(size(fold, 1))")
 #   println("Participantes: $(unique(fold[:, :subject]))")
end


### 6. Normalizar usando MinMaxScaler

In [ ]:
function MinMaxScaler(columna)
    dt = fit(UnitRangeTransform, (columna))
    return StatsBase.transform(dt, columna)
end

for i ∈ 1:length(fold_data)
    for col in names(fold_data[i])
        if eltype(fold_data[i][:,col]) == Float64
            fold_data[i][:,col] = MinMaxScaler(fold_data[i][:,col])
        end
    end
    println("Normalizado el fold $i")
end

---

## Creación de los modelos básicos

### 7. Filtrado

#### 7.1 ANOVA

In [ ]:
using Distributions

function anova(fold,α) # diapositiva 26
    
    # División de Datos en Grupos
    
    grupos = DataFrame[] 
    for clase in clases
        push!(grupos,filter(fila -> fila[clase] == 1,fold))
    end
    println(typeof(grupos[1]))
    
    # Cálculo de la Variabilidad
    
    medias = Array{Float64}(undef, 0, size(grupos[1],2)) 
    for grupo in grupos
        media_grupo = Float64[]
        for col in names(grupo)        
            push!(media_grupo, mean(grupo[!,col]))
        end
        medias = vcat(medias, transpose(media_grupo))
    end 
    println(size(medias))
    medias_entre_grupos = [mean(medias[:][i]) for i in 1:size(medias,2)]
    
    # usaremos implicitamente el hecho de que los grupos tienen el mismo numero de elementos
    # notación de de https://en.wikipedia.org/wiki/One-way_analysis_of_variance
    
    Sb = [sum((medias[:,i] .-  medias_entre_grupos[i]).^2) for i in 1:size(medias,2)]

    Sw = zeros(size(medias,2))
    for (i, grupo) in enumerate(grupos) 
        for (j, col) in enumerate(names(grupo))
            Sw[j] += sum((grupo[!,col] .- medias[i,j]).^2)
        end
    end
    
    # Cálculo del Estadístico F
    
    F = (Sb./(length(clases)-1)) ./ (Sw./(size(fold,1)-length(clases)))
    
    # Determinación de la Significancia
    
    distribucionF = FDist(length(clases)-1, size(fold,1)-length(clases))
    indices = BitVector(F .< quantile.(distribucionF,α))
    indices[end-6:end].=1
    return indices
end

for i in 1:length(fold_data)
    indices = anova(fold_data[i],0.95)
    print(indices) # vemos con un cero las variables que se dejan fuera
    fold_data[i] = fold_data[i][!,indices]
end

In [ ]:
fold_data # vemos que ahora hay menos columnas

#### 7.2 Mutual Information

In [ ]:
function kargmax(v, n) 
    indices_menor_a_mayor = partialsortperm(v, 1:length(v))
    #print(indices_menor_a_mayor)
    return sort(last(indices_menor_a_mayor,n))
end
x = [3,6,2,7,4,5,1,4]
kargmax(x, 4)

In [ ]:
 using InformationMeasures

for i ∈ 1:length(fold_data) # para cada fold
    explicativas = []
    for x ∈ names(fold_data[i]) # por cada variable explicativa
        if eltype(fold_data[i][!,x]) == Float64
             # por cada variable respuesta
            ix = mean([get_mutual_information(fold_data[i][!,x], fold_data[i][!,y]) for y ∈ last(names(fold_data[i]),6)])
            push!(explicativas, ix)
            #println("Fold $i, variable $x, informacion = $ix")
        end
    end
    #println(explicativas)
    indices = kargmax(explicativas,58) # cogemos las 64 variables más explicativas
    for respuesta ∈ length(names(fold_data[i]))-6:length(names(fold_data[i]))
        if respuesta ∉ indices
            push!(indices, respuesta)
        end
    end
    fold_data[i] = fold_data[i][!,indices]
end


In [ ]:
fold_data # vemos que ahora hay 64 columnas

#### 7.3 RFE con el método de LogisticRegression con una eliminación del 50% de las variables en cada pasada.

In [ ]:
using GLM

for i ∈ 1:length(fold_data) # para cada fold
    #fm = @formula(fold_data[i][!,Activity_WALKING] ~  fold_data[i][!,tBodyAcc-mean()-X] )
    #logit = glm(fm, train, Binomial(), ProbitLink())
    explicativas = []
    respuesta = []
    for col in names(fold_data[i])
        if eltype(fold_data[i][!,col]) == Float64
            push!(explicativas,col)
        elseif eltype(fold_data[i][!,col]) == Bool
            push!(respuesta,col)
        end
    end
 
    y = Term(Symbol(respuesta[1]))
    x =  +((Term(Symbol(i)) for i ∈ explicativas)...) # https://discourse.julialang.org/t/non-call-expression-encountered/90725/2
    formula = y ~  x
    logistica = glm(formula, fold_data[i], Binomial(), ProbitLink())
    print(logistica)
end


### 8. Reducción dimensionalidad

#### 8.1 PCA

#### 8.2 LDA

#### 8.3 ICA

#### 8.4 Isomap

#### 8.5 LLE

### 9. Clasificadores

#### 9.1 MLP con al menos las siguientes arquitecturas: [50], [100] [100, 50]

#### 9.2 KNN con valores de vecindario entre 1, 10 y 20

#### 9.3 SVM con el parámetro C con valores 0.1, 0.5 y 1.0

---

## Creación de los modelos ensemble

---

## Conclusiones